In [1]:
# source: https://ogb.stanford.edu/docs/linkprop/#ogbl-collab
import numpy as np
from ogb.linkproppred import LinkPropPredDataset

dataset = LinkPropPredDataset(name = 'ogbl-collab')

split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0]


Using backend: pytorch[21:05:45] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/jimmyzxj/miniconda3/envs/py39/lib/python3.9/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory



In [2]:
node_feats = graph['node_feat'] # (235868, 128)
edge_year = graph['edge_year']
edge_weight = graph['edge_weight'].reshape(-1,) # (2358104, )
edge = graph['edge_index'].T # (2358104, 2)
neg_edge = np.concatenate([valid_edge['edge_neg'], test_edge['edge_neg']])
node_list = np.ones((1, graph['num_nodes']))
edge_list = np.ones((1, edge.shape[0]))
edge_id = np.arange(0, edge.shape[0])

data = {
    "node_feats": node_feats,
    "edge_year": edge_year,
    "edge_weight":edge_weight,
    "edge": edge,
    "neg_edge": neg_edge,
    "node_list": node_list,
    "edge_list": edge_list,
    "edge_id": edge_id
}

In [3]:
np.savez_compressed("ogbl-collab.npz", **data)

In [ ]:
task_data = {
    # use edges (N, 2) to save negative edges
    "val_neg": valid_edge['edge_neg'],
    "test_neg": test_edge['edge_neg'],
    "train_time_window": (1963, 2017),
    "valid_time_window": (2018, 2018),
    "test_time_window": (2019, 2019)
}
np.savez_compressed("ogbl-collab_task_prestore_neg.npz", **task_data)




In [ ]:
task_data_no_pre_stored_neg_edge = {
    "train_time_window": (1963, 2017), # window of (edge_year in train)
    "valid_time_window": (2018, 2018),
    "test_time_window": (2019, 2019)
}
np.savez_compressed("ogbl-collab_task_runtime_sampling.npz", **task_data_no_pre_stored_neg_edge)

In [ ]:
# 1. pre-store neg - index
# 2. no pre-store - time